In [23]:
!pip install pyvirtualdisplay Pillow


In [1]:
import os
import torch
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import gymnasium as gym
from tqdm import tqdm
from dotenv import load_dotenv
from gymnasium import spaces
import matplotlib.patches as patches

SUMO_HOME = 'C:\\Program Files (x86)\\Eclipse\\Sumo'

os.environ['SUMO_HOME'] = SUMO_HOME

# Print the SUMO_HOME environment variable to verify
print("SUMO_HOME is set to:", os.environ.get('SUMO_HOME'))

# Check if the SUMO_HOME path exists
print("Does SUMO_HOME exist?", os.path.exists(os.environ.get('SUMO_HOME')))


# Define file paths
net_file = 'C:/Users/ASUS/Sumo/2024-11-03-18-23-47/osm.net.xml/osm.net.xml'
route_file = 'C:/Users/ASUS/Sumo/2024-11-03-18-23-47/osm.passenger.rou.xml'
out_csv_name = 'C:/Users/ASUS/Sumo/2024-11-03-18-23-47/osm.passenger.csv'
config_file = 'C:/Users/ASUS/Sumo/2024-11-03-18-23-47/osm.sumocfg'


SUMO_HOME is set to: C:\Program Files (x86)\Eclipse\Sumo
Does SUMO_HOME exist? True


In [ ]:
import gymnasium as gym
import sumo_rl
import os
import traci

class Environment:
    def __init__(self, env_name, route_file, net_file, out_csv_name, render_mode='human'):
        self.env_name = env_name
        self.use_gui = True if render_mode == 'human' else False
        self.route_file = route_file
        self.net_file = net_file
        self.out_csv_name = out_csv_name
        self.env = gym.make(
                        env_name,
                        net_file=net_file,
                        route_file=route_file,
                        out_csv_name=out_csv_name,
                        use_gui=self.use_gui,
                        num_seconds=100000
                    )
        self.state, _ = self.env.reset()
        self.done = False
        self.observation_space = self.env.observation_space.shape[0]
        self.action_space = self.env.action_space
    
    def reset(self):
        self.state, _ = self.env.reset()
        self.done = False
        return self.state
    
    def custom_reward(self):
         # Example metrics; you can adjust these as per your objectives
        total_waiting_time = sum([traci.edge.getWaitingTime(edge) for edge in traci.edge.getIDList()])
        queue_length = sum([traci.edge.getLastStepHaltingNumber(edge) for edge in traci.edge.getIDList()])

        # Define custom reward: negative waiting time and queue length to incentivize smooth flow
        reward = -total_waiting_time - queue_length
        return reward
    
    def step(self, action):
        next_state, _, terminated, truncated, info = self.env.step(action)

        reward = self.custom_reward()

        self.state = next_state
        self.done = terminated
        return next_state, reward, self.done or truncated
    
    def render(self):
        self.env.render()
    
    def close(self):
        try:
            self.env.close()
            if traci.isLoaded():
                traci.close()
            print("Env and Traci closed successfully.")
        except Exception as e:
            print("Error while closing the environment:", e)
    
    def get_state(self):
        return self.state
    




## Running Environment (Simple Intersection)

In [3]:
# Define paths to the network and route files (Buffalo Road Intersection)

# net_file = 'C:/Users/ASUS/Sumo/2024-11-03-18-23-47/osm.net.xml/osm.net.xml'
# route_file = 'C:/Users/ASUS/Sumo/2024-11-03-18-23-47/osm.passenger.rou.xml'
# out_csv_name = 'C:/Users/ASUS/Sumo/2024-11-03-18-23-47/osm.passenger.csv'


# Using Custom simple Traffic Intersection
nets_dir = 'nets'

file_name = 'single_intersection_simple'

nets_file = os.path.join(nets_dir, f'{file_name}.net.xml')
routes_file = os.path.join(nets_dir, f'{file_name}.rou.xml')

file_exists = lambda file_path: os.path.exists(file_path)

if not file_exists(nets_file):
    raise FileNotFoundError(f"Net file not found: {net_file}")
if not file_exists(routes_file):
    raise FileNotFoundError(f"Route file not found: {route_file}")


# Instantiate the environment
sumo_env = Environment('sumo-rl-v0', net_file=nets_file, route_file=routes_file, out_csv_name=out_csv_name, render_mode='human')

print("Observation Space:", sumo_env.observation_space)
print("Action Space:", sumo_env.action_space)
print("Initial State:", sumo_env.state)

max_steps = 200

try:
    print("action space", sumo_env.action_space)
    for step in range(max_steps):
        sumo_env.render()
        action = sumo_env.env.action_space.sample()
        next_state, reward, done = sumo_env.step(action)
        print("Action ", action, " Reward ", reward, " Done ", done)
        
        if done:
            sumo_env.reset()
            break

except KeyboardInterrupt:
    print("Keyboard interrupt detected. Closing the environment.")

finally:
    sumo_env.close()

Observation Space: 11
Action Space: 2
Initial State: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
action space 2
Action  1  Reward  -0.0  Done  False
Action  1  Reward  -0.0  Done  False
Action  0  Reward  -3.0  Done  False
Action  0  Reward  -25.0  Done  False


c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\gymnasium\utils\passive_env_checker.py:335: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


Action  1  Reward  -14.0  Done  False
Action  0  Reward  -0.0  Done  False
Action  1  Reward  -2.0  Done  False
Action  1  Reward  -12.0  Done  False
Action  1  Reward  -33.0  Done  False
Action  1  Reward  -53.0  Done  False
Action  1  Reward  -82.0  Done  False
Action  0  Reward  -0.0  Done  False
Action  1  Reward  -6.0  Done  False
Action  1  Reward  -8.0  Done  False
Action  0  Reward  -24.0  Done  False
Action  1  Reward  -39.0  Done  False
Action  1  Reward  -60.0  Done  False
Action  0  Reward  -0.0  Done  False
Action  1  Reward  -10.0  Done  False
Action  1  Reward  -0.0  Done  False
Action  0  Reward  -4.0  Done  False
Action  1  Reward  -13.0  Done  False
Action  0  Reward  -3.0  Done  False
Action  1  Reward  -24.0  Done  False
Action  1  Reward  -10.0  Done  False
Action  1  Reward  -7.0  Done  False
Action  0  Reward  -12.0  Done  False
Action  1  Reward  -55.0  Done  False
Action  1  Reward  -50.0  Done  False
Action  1  Reward  -11.0  Done  False
Action  1  Reward  -14

In [1]:
import gymnasium as gym
from sumo_rl import SumoEnvironment
import os

# Print SUMO_HOME for confirmation
print("SUMO_HOME is set to:", os.environ['SUMO_HOME'])

# Initialize SUMO Gym environment
env = gym.make(
    'sumo-rl-v0',
    net_file=net_file,
    route_file=route_file,
    out_csv_name=out_csv_name,
    use_gui=True,
    num_seconds=100000
)

# Reset the environment and start simulation
obs, info = env.reset()
done = False

print(env.action_space)

while not done:
    next_obs, reward, terminated, truncated, info = env.step(env.action_space.sample())
    print("Reward:", reward)
    print("Observation:", next_obs)
    done = terminated or truncated


KeyboardInterrupt: 

In [9]:
import sumolib

# Load the network file
net = sumolib.net.readNet(nets_file)  # Replace with your network file

# Get all edge IDs
edge_ids = [edge.getID() for edge in net.getEdges()]
print("Edge IDs:", edge_ids)

# List all traffic light IDs
tls_ids = [tls.getID() for tls in net.getTrafficLights()]
print("Traffic Light IDs:", tls_ids)

# # Get all vehicle IDs in the current step
# vehicle_ids = traci.vehicle.getIDList()
# print("Vehicle IDs:", vehicle_ids)




Edge IDs: ['n_t', 't_e', 't_s', 'w_t']
Traffic Light IDs: ['t']


In [ ]:
import traci
import sumolib
import sys
import time

try:
    # Define paths to SUMO binary, network file, and route file
    sumo_binary = "sumo-gui"  # Use "sumo" for non-GUI mode
    # Command to start SUMO with TraCI connection
    sumo_cmd = [sumo_binary, "-n", nets_file, "-r", routes_file, "--start"]
    
    # Attempt to start SUMO with the specified command
    try:
        traci.start(sumo_cmd, label="sim1")
    except traci.exceptions.TraCIException as e:
        print(f"Failed to start TraCI: {e}")
        sys.exit(1)

    print("TraCI connected successfully.")
    
    # Main simulation loop with safe TraCI operations
    for step in range(1000):
        try:
            traci.simulationStep()  # Advance simulation by one step
            
            # Check vehicle count on a specific edge
            edge_id = "t_e"  # Replace with actual edge ID
            try:
                vehicle_count = traci.edge.getLastStepVehicleNumber(edge_id)
                print(f"Step {step}, Vehicle count on {edge_id}: {vehicle_count}")
            except traci.exceptions.TraCIException as e:
                print(f"Error retrieving vehicle count for edge {edge_id}: {e}")
            
            # Get traffic light phase and queue length
            tls_id = "t"  # Replace with actual traffic light ID
            try:
                phase = traci.trafficlight.getPhase(tls_id)
                print(f"Traffic light {tls_id} phase: {phase}")
            except traci.exceptions.TraCIException as e:
                print(f"Error retrieving traffic light phase for {tls_id}: {e}")
            
            # Pause to inspect values for each step
            # input("Press Enter to continue to the next step...")
            time.sleep(0.5)
            
            if step >= max_steps -1:
                break            

        except KeyboardInterrupt:
            print("\nSimulation interrupted by user.")
            break  # Safely exit the loop if user interrupts

except (FileNotFoundError, traci.exceptions.TraCIException) as e:
    print(f"Error in setup or simulation: {e}")

except Exception as e:
    print(f"Unexpected error occurred: {e}")

finally:
    # Ensure that TraCI closes safely even if an error occurs
    try:
        if traci.isLoaded():
            traci.close()
            print("TraCI connection closed safely.")
    except traci.exceptions.TraCIException as e:
        print(f"Error while closing TraCI: {e}")


TraCI connected successfully.
Step 0, Vehicle count on t_e: 0
Traffic light t phase: 0
Step 1, Vehicle count on t_e: 0
Traffic light t phase: 0
Step 2, Vehicle count on t_e: 0
Traffic light t phase: 0
Step 3, Vehicle count on t_e: 0
Traffic light t phase: 0
Step 4, Vehicle count on t_e: 0
Traffic light t phase: 0
Step 5, Vehicle count on t_e: 0
Traffic light t phase: 0
Step 6, Vehicle count on t_e: 0
Traffic light t phase: 0
Step 7, Vehicle count on t_e: 0
Traffic light t phase: 0
Step 8, Vehicle count on t_e: 0
Traffic light t phase: 0
Step 9, Vehicle count on t_e: 0
Traffic light t phase: 0
Step 10, Vehicle count on t_e: 0
Traffic light t phase: 0
Step 11, Vehicle count on t_e: 0
Traffic light t phase: 0
Step 12, Vehicle count on t_e: 0
Traffic light t phase: 0
Step 13, Vehicle count on t_e: 0
Traffic light t phase: 0
Step 14, Vehicle count on t_e: 0
Traffic light t phase: 0
Step 15, Vehicle count on t_e: 0
Traffic light t phase: 0
Step 16, Vehicle count on t_e: 0
Traffic light t pha

In [3]:
import os
os.environ["PATH"] += os.pathsep + r"C:\Program Files (x86)\Eclipse\Sumo\bin"

!sumo


Eclipse SUMO sumo Version 1.21.0
 Build features: Windows-10.0.17763 AMD64 MSVC 19.29.30133.0 Release FMI Proj GUI Intl SWIG Eigen
 Copyright (C) 2001-2024 German Aerospace Center (DLR) and others; https://sumo.dlr.de
 License EPL-2.0: Eclipse Public License Version 2 <https://eclipse.org/legal/epl-v20.html>
 Use --help to get the list of options.


## Trial of Custom Environment without sumoRL

In [39]:
import os
import time
import traci
import psutil
import gymnasium as gym
import numpy as np
from gymnasium import spaces

def kill_sumo_process():
    """Kill any running SUMO processes."""
    os.system('taskkill /f /im sumo.exe')
    time.sleep(1)  # Wait for the process to terminate

    # Check if the SUMO process is still running
    for proc in psutil.process_iter(['pid', 'name']):
        if 'sumo.exe' in proc.info['name'].lower():
            print("SUMO process is still running.")
            return False
    print("SUMO process was successfully terminated.")
    return True

class TrafficLightSumoEnv(gym.Env):
    def __init__(self, net_file, route_file, config_file, use_gui=False):
        super(TrafficLightSumoEnv, self).__init__()

        # Initialize SUMO parameters
        self.net_file = net_file
        self.route_file = route_file
        self.config_file = config_file
        self.use_gui = use_gui

        # Define observation and action spaces for the traffic light agent
        self.observation_space = spaces.Box(low=0, high=1, shape=(4,), dtype=np.float32)  # Example observation space
        self.action_space = spaces.Discrete(4)  # Four actions for the traffic light (e.g., Red, Green, Yellow, None)

        # Initialize SUMO simulation
        self._init_sumo()

    def _init_sumo(self):
        """Initialize the SUMO simulation."""
        print(f"Config file: {self.config_file}")
        print(f"Network file: {self.net_file}")
        print(f"Route file: {self.route_file}")

        try:
            traci.close()  # Ensure any previous connections are closed
            print("TraCI connection closed.")
        except Exception as e:
            print(f"Error closing TraCI: {e}")

        # Kill any lingering SUMO processes before starting a new one
        if not kill_sumo_process():
            print("Error: Unable to terminate SUMO. Please try manually.")
            return

        sumo_binary = "sumo-gui" if self.use_gui else "sumo"
        command = [sumo_binary, "-c", self.config_file, "-n", self.net_file, "-r", self.route_file, "--quit-on-end", "--start-port", "8811"]
        print("Starting SUMO with command:", command)

        try:
            traci.start(command)  # Start SUMO
            print("SUMO started successfully.")
            traci.connect(port=8811)  # Connect to SUMO via the specified port
            print("Connected to SUMO on port 8811.")
        except Exception as e:
            print(f"Error starting or connecting to SUMO: {e}")

    def reset(self):
        """Reset the environment."""
        try:
            traci.load([self.net_file, self.route_file])
            obs = self._get_observations()
            return obs
        except Exception as e:
            print(f"Error during reset: {e}")
            return None

    def _get_observations(self):
        """Get observations for the traffic light agent (i.e., traffic state around the light)."""
        vehicle_ids = traci.vehicle.getIDList()
        vehicle_speeds = [traci.vehicle.getSpeed(vehicle_id) for vehicle_id in vehicle_ids]

        avg_speed = np.mean(vehicle_speeds) if vehicle_speeds else 0.0
        vehicle_count = len(vehicle_ids)

        obs = np.array([avg_speed, vehicle_count, 1.0, 0.0], dtype=np.float32)
        return obs

    def step(self, action):
        """Take a step in the environment by updating the traffic light state."""
        try:
            if action == 0:  # Red light
                traci.trafficlight.setRedYellowGreenState('traffic_light', 'r')
            elif action == 1:  # Green light
                traci.trafficlight.setRedYellowGreenState('traffic_light', 'g')
            elif action == 2:  # Yellow light
                traci.trafficlight.setRedYellowGreenState('traffic_light', 'y')
            elif action == 3:  # No action (maintain current state)
                pass

            traci.simulationStep()  # Advance the simulation by one step
            obs = self._get_observations()
            rewards = self._compute_rewards()
            done = False  # Define your termination condition here
            return obs, rewards, done, {}
        except Exception as e:
            print(f"Error during step: {e}")
            return None, None, True, {}

    def _compute_rewards(self):
        """Compute rewards for the traffic light agent based on traffic flow and wait times."""
        rewards = 0.0
        vehicle_ids = traci.vehicle.getIDList()
        vehicle_wait_times = [traci.vehicle.getWaitingTime(vehicle_id) for vehicle_id in vehicle_ids]
        vehicle_speeds = [traci.vehicle.getSpeed(vehicle_id) for vehicle_id in vehicle_ids]

        if vehicle_speeds:
            avg_speed = np.mean(vehicle_speeds)
            rewards += avg_speed * 0.5

        if vehicle_wait_times:
            avg_wait_time = np.mean(vehicle_wait_times)
            rewards -= avg_wait_time * 0.5  # Penalize high waiting time

        return rewards

    def close(self):
        """Close the SUMO simulation and ensure proper connection cleanup."""
        try:
            traci.close()  # Explicitly close the connection
            print("TraCI connection closed.")
        except traci.TraCIException:
            print("No active TraCI connection to close.")

# Example usage
# env = TrafficLightSumoEnv("network.net.xml", "routes.rou.xml", "config.sumocfg")

# Ensure to call env.close() when you're done with the environment.


In [28]:
import os
import psutil
import time

def kill_sumo_process():
    # Kill any running SUMO processes
    os.system('taskkill /f /im sumo.exe')
    
    # Wait for a few seconds to allow the process to terminate
    time.sleep(1)

    # Check if the SUMO process is still running
    for proc in psutil.process_iter(['pid', 'name']):
        if 'sumo.exe' in proc.info['name'].lower():
            print("SUMO process is still running.")
            return False
    print("SUMO process was successfully terminated.")
    return True

def start_sumo_simulation(sumo_cmd):
    # Ensure no leftover SUMO process
    if not kill_sumo_process():
        print("Error: Unable to terminate SUMO. Please try manually.")
        return
    
    # Start the SUMO simulation
    try:
        traci.start(sumo_cmd)
    except Exception as e:
        print(f"Error starting SUMO: {e}")
        return

# Example command to start SUMO (adjust with your actual files)
sumo_cmd = ["sumo", "-c", "config.sumocfg", "-n", "network.net.xml", "-r", "routes.rou.xml"]

start_sumo_simulation(sumo_cmd)



SUMO process was successfully terminated.
Error starting SUMO: Connection 'default' is already active.


In [ ]:
env = TrafficLightSumoEnv(net_file, route_file, config_file ,use_gui=True)

Config file: C:/Users/ASUS/Sumo/2024-11-03-18-23-47/osm.sumocfg
Network file: C:/Users/ASUS/Sumo/2024-11-03-18-23-47/osm.net.xml/osm.net.xml
Route file: C:/Users/ASUS/Sumo/2024-11-03-18-23-47/osm.passenger.rou.xml
Error closing TraCI: [WinError 10054] An existing connection was forcibly closed by the remote host
SUMO process was successfully terminated.
Starting SUMO with command: ['sumo', '-c', 'C:/Users/ASUS/Sumo/2024-11-03-18-23-47/osm.sumocfg', '-n', 'C:/Users/ASUS/Sumo/2024-11-03-18-23-47/osm.net.xml/osm.net.xml', '-r', 'C:/Users/ASUS/Sumo/2024-11-03-18-23-47/osm.passenger.rou.xml', '--quit-on-end', '--start-port', '8811']
Error starting or connecting to SUMO: Connection 'default' is already active.


In [26]:
import psutil

def check_sumo_running():
    # Check for SUMO process
    for proc in psutil.process_iter(['pid', 'name']):
        if 'sumo.exe' in proc.info['name'].lower():
            return True  # SUMO is still running
    return False  # SUMO is not running

# Kill any running SUMO processes
os.system('taskkill /f /im sumo.exe')

# Wait for a moment and check if SUMO is still running
if check_sumo_running():
    print("SUMO process is still running.")
else:
    print("SUMO process was successfully terminated.")

SUMO process was successfully terminated.
